# Table of Contents
 <p>

In this practice we will perform topic modeling and sentiment analysis on two datasets.

**Activity 1:** Load the kaggle voted dataset and perform following tasks 

* Focus on the "Description" column and preprocess if required
* Fit an LDA model with 10 topics on the "Description" column 
    * While creating TF matrix ignore terms that have a document frequency strictly higher than 100
* Print top-5 words per topic

### Load data file

In [1]:
# load necessary packages
import json
import pandas as pd
import re

In [2]:
filepath = "/dsa/data/DSA-8410/voted-kaggle-dataset.csv"

df = pd.read_csv(filepath, encoding = 'utf-8')
print(df['Description'].head(5))

0    The datasets contains transactions made by cre...
1    The ultimate Soccer database for data analysis...
2    Background\nWhat can we say about the success ...
3    Context\nInformation on more than 170,000 Terr...
4    Context\nBitcoin is the longest running and mo...
Name: Description, dtype: object


### Create a TF matrix

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation


In [4]:
print(df.shape)
desc = df['Description'].dropna()
print(desc.shape)

(2150, 15)
(2145,)


In [5]:
docs = desc.values
countVectorizer = CountVectorizer(stop_words='english', max_df=100 )
termFrequency = countVectorizer.fit_transform(docs)
featureNames = countVectorizer.get_feature_names()

### Fit an LDA model

In [6]:
lda = LatentDirichletAllocation(n_components=10)
lda.fit(termFrequency)    

LatentDirichletAllocation()

### Print top 10 words per topic

In [7]:
for idx, topic in enumerate(lda.components_):
    print("Topic ", idx, " ".join(featureNames[i] for i in topic.argsort()[:-10 - 1:-1]))

Topic  0 dry speech nominal post string news uk audio dub lub
Topic  1 integer numeric reviews strongly review interested enjoy agree disagree product
Topic  2 txt race movie air survey horse rating species movies standard
Topic  3 player food question survey shot census goals wine missing ice
Topic  4 player players pokemon games season stats sports movies football school
Topic  5 trained police children station child prices college market bitcoin weather
Topic  6 en float software percent votes acquired les wouldn la citations
Topic  7 da policies kumar documento que education insurance pharmacy guard 01
Topic  8 trained tweets tweet twitter residual articles south languages deep station
Topic  9 crime risk police election stanford rates application traffic position registration


**Activity 2**: Load the Twitter US Airline Sentiment data and perform the following task

* Preprocess the text column with the removal of all the mentions 
* Identify sentiment (pos, neg, neu) for each of the tweets using the standard rule mentioned in the lab
* Print the classification report

### Load data file

In [8]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import classification_report

In [9]:
filepath = "/dsa/data/DSA-8410/Twitter-US-Airline-Sentiment/Tweets.csv"
# filepath = "../../../../data/Twitter-US-Airline-Sentiment/Tweets.csv"

df_all= pd.read_csv(filepath)
df_all.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


### Preprocess the tweets

In [10]:
tweets = [re.sub(r'@(\w+)', ' ', t) for t in df_all['text'].values]

### Identify polarity for each tweet 

In [11]:
analyzer = SentimentIntensityAnalyzer()
tweets_sentiment = [analyzer.polarity_scores(t) for t in tweets]

df = pd.DataFrame(tweets_sentiment)
df['tweet'] = tweets
df.head()

,neg,neu,pos,compound,tweet
0,0.000,1.000,0.0,0.0000,What said.
1,0.000,1.000,0.0,0.0000,plus you've added commercials to the experie...
2,0.000,1.000,0.0,0.0000,I didn't today... Must mean I need to take a...
3,0.259,0.741,0.0,-0.5984,"it's really aggressive to blast obnoxious ""e..."
4,0.351,0.649,0.0,-0.5829,and it's a really big bad thing about it


In [12]:
df.describe()

,neg,neu,pos,compound
count,14640.000000,14640.000000,14640.000000,14640.000000
mean,0.089417,0.777355,0.133227,0.052397
std,0.120441,0.187613,0.181640,0.456590
min,0.000000,0.000000,0.000000,-0.966800
25%,0.000000,0.672000,0.000000,-0.296000
50%,0.000000,0.797000,0.072000,0.000000
75%,0.156000,0.913000,0.210000,0.437400
max,1.000000,1.000000,1.000000,0.976000


### Perform a rule-base classification

In [13]:
df['sentiment'] = 'NEU'
df.loc[df['compound'] > 0.05, 'sentiment'] = 'POS'
df.loc[df['compound'] < -0.05, 'sentiment'] = 'NEG'

df.head()

,neg,neu,pos,compound,tweet,sentiment
0,0.000,1.000,0.0,0.0000,What said.,NEU
1,0.000,1.000,0.0,0.0000,plus you've added commercials to the experie...,NEU
2,0.000,1.000,0.0,0.0000,I didn't today... Must mean I need to take a...,NEU
3,0.259,0.741,0.0,-0.5984,"it's really aggressive to blast obnoxious ""e...",NEG
4,0.351,0.649,0.0,-0.5829,and it's a really big bad thing about it,NEG


In [14]:
import seaborn as sns
sns.set()
sns.boxplot(x="sentiment", y="compound", data=df);

### Report classification metrices

In [15]:
y_true = df_all["airline_sentiment"].map({'neutral': 'NEU', 'positive': 'POS', 'negative': 'NEG'})
y_pred = df['sentiment']
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         NEG       0.90      0.50      0.64      9178
         NEU       0.38      0.43      0.40      3099
         POS       0.34      0.87      0.49      2363

    accuracy                           0.54     14640
   macro avg       0.54      0.60      0.51     14640
weighted avg       0.70      0.54      0.57     14640



# Save your notebook, then `File > Close and Halt`